In [5]:
# =========================
# AOI PREVIEW (nose from segmentation; no saving)
# =========================
# Requires:
#   pip install face_recognition pillow numpy scipy
# Optional (for notebook preview):
#   pip install matplotlib

import os, math
import numpy as np
from PIL import Image, ImageDraw
import face_recognition
from scipy.io import loadmat
from scipy.spatial import ConvexHull

# =========================
# Config (set to your paths)
# =========================
IMG_PATH = "/Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ET Training (Brave)/Indonesian faces/BRAVE_adultF_adultF_1.jpg"
PRED_DIR = "/Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ETbrave"
PRED_SUFFIX = "-pred.mat"   # produced by your pred_folder.py

SHOW_MODE = "pil"           # "pil" or "matplotlib"

# AOI parameters
DIVIDER_OFFSET_Y = -60
MOUTH_OFFSET_Y = 8
NOSE_DIVIDER_OFFSET_Y = -30
OUTLINE_SCALE = 1

# segmentation label mapping: 10 = nose (repo's label set)
NOSE_LABEL = 10

# tweak maps you used before (kept as-is)
FOREHEAD_TWEAKS_FACE_LEFT  = {0:(+2,0), 1:(-9,0),  2:(-17,0), 3:(-24,0), 4:(-2,0), 5:(-3,0), 6:(-5,0)}
FOREHEAD_TWEAKS_FACE_RIGHT = {0:(+2,0), 1:(-13,0), 2:(-17,0), 3:(-24,0), 4:(-2,0), 5:(-3,0), 6:(-5,0)}
CHIN_SCALE_FACE_LEFT =  {2:0.95, 3:0.91, 4:0.91, 5:0.80, 6:0.80, 7:0.80, 8:0.80, 9:0.90, 10:0.85, 11:0.85, 12:0.90, 13:0.93}
CHIN_SCALE_FACE_RIGHT = {0:0.96,1:0.96,2:0.90,3:0.90,4:0.83,5:0.83,6:0.80,7:0.80,8:0.80,9:0.90,10:0.88,11:0.88,12:0.88,13:0.95,15:1.03,16:1.00}

# =========================
# Helpers
# =========================
def bezier_quad(p0, p1, p2, n_points=10):
    out = []
    for i in range(n_points):
        t = i/(n_points-1)
        x = (1-t)**2*p0[0] + 2*(1-t)*t*p1[0] + t**2*p2[0]
        y = (1-t)**2*p0[1] + 2*(1-t)*t*p1[1] + t**2*p2[1]
        out.append((int(round(x)), int(round(y))))
    return out

def get_bbox(coords):
    xs = [x for x, y in coords]; ys = [y for x, y in coords]
    return min(xs), min(ys), max(xs), max(ys)

def enlarge_polygon(polygon, scale_factor):
    cx = sum(x for x, _ in polygon)/len(polygon)
    cy = sum(y for _, y in polygon)/len(polygon)
    return [(int(round((x-cx)*scale_factor+cx)), int(round((y-cy)*scale_factor+cy))) for x, y in polygon]

def clip_polygon_with_rect(poly, x_min, y_min, x_max, y_max):
    def clip_edge(points, inside, intersect):
        if not points: return []
        out, s = [], points[-1]
        for e in points:
            if inside(e):
                if inside(s): out.append(e)
                else: out.append(intersect(s, e)); out.append(e)
            else:
                if inside(s): out.append(intersect(s, e))
            s = e
        return out
    def inside_left(p):   return p[0] >= x_min
    def inside_right(p):  return p[0] <= x_max
    def inside_top(p):    return p[1] >= y_min
    def inside_bottom(p): return p[1] <= y_max
    def intersect_vert(p1, p2, xk):
        x1,y1=p1; x2,y2=p2
        if x2==x1: return (xk,y1)
        t=(xk-x1)/(x2-x1); y=y1 + t*(y2-y1); return (xk,y)
    def intersect_horiz(p1, p2, yk):
        x1,y1=p1; x2,y2=p2
        if y2==y1: return (x1,yk)
        t=(yk-y1)/(y2-y1); x=x1 + t*(x2-x1); return (x,yk)
    pts = [(float(x),float(y)) for x,y in poly]
    pts = clip_edge(pts, inside_left,  lambda a,b: intersect_vert(a,b,x_min))
    pts = clip_edge(pts, inside_right, lambda a,b: intersect_vert(a,b,x_max))
    pts = clip_edge(pts, inside_top,   lambda a,b: intersect_horiz(a,b,y_min))
    pts = clip_edge(pts, inside_bottom,lambda a,b: intersect_horiz(a,b,y_max))
    return [(int(round(x)), int(round(y))) for x,y in pts]

def largest_cc(mask_bool):
    h, w = mask_bool.shape
    visited = np.zeros_like(mask_bool, dtype=bool)
    best, best_idxs = 0, None
    nbrs = [(1,0),(-1,0),(0,1),(0,-1)]
    for y in range(h):
        for x in range(w):
            if mask_bool[y,x] and not visited[y,x]:
                comp, stack = [], [(y,x)]
                visited[y,x] = True
                while stack:
                    cy,cx = stack.pop(); comp.append((cy,cx))
                    for dy,dx in nbrs:
                        ny,nx = cy+dy,cx+dx
                        if 0<=ny<h and 0<=nx<w and mask_bool[ny,nx] and not visited[ny,nx]:
                            visited[ny,nx]=True; stack.append((ny,nx))
                if len(comp)>best: best=len(comp); best_idxs=comp
    out = np.zeros_like(mask_bool, dtype=bool)
    if best_idxs:
        ys, xs = zip(*best_idxs); out[ys, xs] = True
    return out

def polygon_from_mask(mask_bool):
    ys, xs = np.nonzero(mask_bool)
    if len(xs) < 3: return []
    pts = np.column_stack([xs, ys])
    try:
        hull = ConvexHull(pts)
        poly = [(int(pts[i,0]), int(pts[i,1])) for i in hull.vertices]
    except Exception:
        poly = [(int(x), int(y)) for x, y in zip(xs, ys)]
    # simple RDP-like reduction: take every k-th point for readability
    if len(poly) > 200:
        step = max(1, len(poly)//200)
        poly = poly[::step]
    return poly

def seg_nose_polygon(pred_mat_path, img_w, img_h, face_poly=None):
    if not os.path.exists(pred_mat_path): return []
    d = loadmat(pred_mat_path)
    pred = None
    for k in ("prediction", "pred", "label"):
        if k in d: pred = d[k]; break
    if pred is None: return []
    pred = np.array(pred).squeeze()
    if pred.shape != (img_h, img_w):
        if pred.T.shape == (img_h, img_w): pred = pred.T
        else: return []
    nose = (pred == NOSE_LABEL)
    if face_poly and len(face_poly)>=3:
        face_mask = Image.new("1", (img_w, img_h), 0)
        ImageDraw.Draw(face_mask).polygon(face_poly, outline=1, fill=1)
        nose &= np.array(face_mask, dtype=bool)
    nose = largest_cc(nose)
    return polygon_from_mask(nose)

def bezier_forehead(left_eyebrow, right_eyebrow, is_left_face):
    offset = 10; widen_offset = 10
    left_ext  = (left_eyebrow[0][0] - widen_offset, left_eyebrow[0][1] + offset)
    right_ext = (right_eyebrow[-1][0] + widen_offset, right_eyebrow[-1][1] + offset)
    ctrl = ((left_ext[0]+right_ext[0])//2, min(left_ext[1], right_ext[1]) - 380)
    pts = bezier_quad(left_ext, ctrl, right_ext, n_points=10)
    tweaks = FOREHEAD_TWEAKS_FACE_LEFT if is_left_face else FOREHEAD_TWEAKS_FACE_RIGHT
    out = []
    for i,(x,y) in enumerate(pts):
        dx,dy = tweaks.get(i,(0,0)); out.append((x+dx, y+dy))
    return out, left_ext, right_ext

# =========================
# Load image, landmarks
# =========================
image = face_recognition.load_image_file(IMG_PATH)
img_w, img_h = image.shape[1], image.shape[0]
faces = face_recognition.face_landmarks(image)
if not faces:
    raise RuntimeError("No faces detected in image.")

def face_center_x(fl): return fl["chin"][8][0]
faces_sorted = sorted(faces, key=face_center_x)

pil = Image.fromarray(image)
draw = ImageDraw.Draw(pil)

# =========================
# Build outline + dividers per face; draw AOIs
# =========================
img_stem = os.path.splitext(os.path.basename(IMG_PATH))[0]
pred_path = os.path.join(PRED_DIR, f"{img_stem}{PRED_SUFFIX}")

for idx, fl in enumerate(faces_sorted, start=1):
    is_left = (idx == 1)
    chin = fl["chin"]; le = fl["left_eyebrow"]; re = fl["right_eyebrow"]

    x_center = chin[8][0]
    scale_map = CHIN_SCALE_FACE_LEFT if is_left else CHIN_SCALE_FACE_RIGHT
    scaled_chin = []
    for i,(x,y) in enumerate(chin):
        s = scale_map.get(i, 1.0)
        scaled_chin.append((int(round((x - x_center)*s + x_center)), y))

    forehead_pts, left_ext, right_ext = bezier_forehead(le, re, is_left)
    outline = scaled_chin + forehead_pts[::-1]

    # dividers
    divider_y = int(round((left_ext[1] + right_ext[1]) / 2))
    bottom_lip = fl.get("bottom_lip", [])
    mouth_y = (max(p[1] for p in bottom_lip) + MOUTH_OFFSET_Y) if bottom_lip else \
              (int(round(np.mean([p[1] for p in chin[6:11]]))) + MOUTH_OFFSET_Y)

    enlarged_poly = enlarge_polygon(outline, OUTLINE_SCALE)
    ex_min, ey_min, ex_max, ey_max = get_bbox(enlarged_poly)

    # horizontal bands
    y_eye_top = max(ey_min, min(divider_y + DIVIDER_OFFSET_Y, ey_max))
    # split between eyes/mouth uses nose tip height if we had landmarks; keep simple:
    # use middle between divider_y and mouth_y, plus offset
    y_mid = max(ey_min, min(int(round((divider_y + mouth_y)/2)) + NOSE_DIVIDER_OFFSET_Y, ey_max))
    y_mouth = max(ey_min, min(mouth_y, ey_max))
    y_eye_top, y_mid = min(y_eye_top, y_mid), max(y_eye_top, y_mid)
    y_mid, y_mouth = min(y_mid, y_mouth), max(y_mid, y_mouth)

    # refine nose from segmentation (masked to this face outline)
    nose_poly = seg_nose_polygon(pred_path, img_w, img_h, face_poly=outline)

    # Draw enlarged face AOI contour (preview)
    draw.line(enlarged_poly + [enlarged_poly[0]], fill="orange", width=2)
    draw.text((ex_min+5, max(0, ey_min-18)), f"Face {idx}", fill="white")

    # Draw NOSE
    if len(nose_poly) >= 3:
        draw.line(nose_poly + [nose_poly[0]], fill="purple", width=3)
        # label
        nx, ny, _, _ = get_bbox(nose_poly)
        draw.text((nx+5, ny+5), f"Nose_F{idx}", fill="purple")
    else:
        draw.text((ex_min+8, ey_min+8), "NO SEG NOSE", fill="purple")

    # Clip helper
    def clip_band(poly, y0, y1):
        return clip_polygon_with_rect(poly, x_min=0, y_min=y0, x_max=img_w, y_max=y1)

    # EYE band (minus nose just for preview border)
    eye_outer = clip_band(enlarged_poly, y_eye_top, y_mid)
    if len(eye_outer) >= 3:
        draw.line(eye_outer + [eye_outer[0]], fill="red", width=2)
        ex, ey, _, _ = get_bbox(eye_outer)
        draw.text((ex+5, ey+5), f"Eye_F{idx}", fill="red")
        if len(nose_poly) >= 3:
            eye_hole = clip_band(nose_poly, y_eye_top, y_mid)
            if len(eye_hole) >= 3:
                draw.line(eye_hole + [eye_hole[0]], fill="red", width=1)

    # MOUTH band (minus nose just for preview border)
    mouth_outer = clip_band(enlarged_poly, y_mid, y_mouth)
    if len(mouth_outer) >= 3:
        draw.line(mouth_outer + [mouth_outer[0]], fill="orange", width=2)
        mx, my, _, _ = get_bbox(mouth_outer)
        draw.text((mx+5, my+5), f"Mouth_F{idx}", fill="orange")
        if len(nose_poly) >= 3:
            mouth_hole = clip_band(nose_poly, y_mid, y_mouth)
            if len(mouth_hole) >= 3:
                draw.line(mouth_hole + [mouth_hole[0]], fill="orange", width=1)

# =========================
# PREVIEW ONLY (no saving)
# =========================
if SHOW_MODE.lower() == "pil":
    pil.show()
else:
    try:
        import matplotlib.pyplot as plt
        plt.figure(); plt.imshow(pil); plt.axis("off"); plt.show()
    except Exception as e:
        print("Matplotlib preview failed, falling back to PIL:", e); pil.show()


In [14]:
# =========================
# Segmentation-only AOI PREVIEW + SAVE (nose normal; eyes/mouth scaled; no eyebrows)
# =========================
# Requires in your 'aoi' env:
#   pip install face_recognition pillow numpy scipy matplotlib

import os, csv
import numpy as np
from PIL import Image, ImageDraw
from scipy.io import loadmat
from scipy.spatial import ConvexHull
import face_recognition

# ---------- CONFIG ----------
IMG_PATH   = "/Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ET Training (Brave)/Indonesian faces/BRAVE_adultF_adultF_1.jpg"
PRED_DIR   = "/Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ETbrave"
OUTPUT_DIR = "/Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ETbrave/aoi_outputs"
PRED_SUFFIX = "-pred.mat"       # produced by pred_folder.py
SHOW_MODE   = "pil"             # "pil" or "matplotlib"

EXPAND_FACE_BOX = 0.15          # expand face rectangle before clipping AOIs

# Per-AOI scaling (use your current numbers)
AOI_SCALE_NOSE  = 1.15          # nose scale (1.0 = no enlarge)
AOI_SCALE_EYES  = 1.15          # eyes scale
AOI_SCALE_MOUTH = 1.15          # mouth scale

INCLUDE_EYEBROWS_IN_EYES = False  # exclude eyebrows from eye AOI

# segmentation labels (per MLT repo)
LBL_EYE = 4
LBL_BROW = 5
LBL_MOUTH = 8
LBL_NOSE = 10

# colors
COLOR_NOSE  = "purple"
COLOR_EYE   = "red"
COLOR_MOUTH = "orange"
TEXT_COLOR  = "white"

# ---------- HELPERS ----------
def clamp(v, lo, hi): return max(lo, min(hi, v))

def expand_box(box, w, h, ratio=0.15):
    top, right, bottom, left = box
    bw, bh = right - left, bottom - top
    dx, dy = int(round(bw * ratio)), int(round(bh * ratio))
    nt = clamp(top - dy,    0, h-1)
    nb = clamp(bottom + dy, 0, h-1)
    nl = clamp(left - dx,   0, w-1)
    nr = clamp(right + dx,  0, w-1)
    return (nt, nr, nb, nl)

def all_cc_masks(mask_bool, min_pixels=80):
    """Return list of boolean masks for each 4-connected component with >= min_pixels."""
    h, w = mask_bool.shape
    seen = np.zeros_like(mask_bool, dtype=bool)
    comps = []
    nbrs = [(1,0),(-1,0),(0,1),(0,-1)]
    for y in range(h):
        for x in range(w):
            if mask_bool[y,x] and not seen[y,x]:
                stack = [(y,x)]
                seen[y,x] = True
                ys, xs = [], []
                while stack:
                    cy, cx = stack.pop()
                    ys.append(cy); xs.append(cx)
                    for dy,dx in nbrs:
                        ny, nx = cy+dy, cx+dx
                        if 0<=ny<h and 0<=nx<w and mask_bool[ny,nx] and not seen[ny,nx]:
                            seen[ny,nx] = True
                            stack.append((ny,nx))
                if len(xs) >= min_pixels:
                    m = np.zeros_like(mask_bool, dtype=bool)
                    m[np.array(ys), np.array(xs)] = True
                    comps.append(m)
    return comps

def polygon_from_mask(mask_bool, max_points=200):
    ys, xs = np.nonzero(mask_bool)
    if len(xs) < 3:
        return []
    pts = np.column_stack([xs, ys])
    try:
        hull = ConvexHull(pts)
        poly = [(int(pts[i,0]), int(pts[i,1])) for i in hull.vertices]
    except Exception:
        poly = [(int(x), int(y)) for x, y in zip(xs, ys)]
    if len(poly) > max_points:
        step = max(1, len(poly)//max_points)
        poly = poly[::step]
    return poly

def enlarge_polygon(poly, scale):
    if len(poly) < 3: return poly
    cx = sum(x for x,_ in poly) / len(poly)
    cy = sum(y for _,y in poly) / len(poly)
    out = []
    for x,y in poly:
        out.append((int(round((x - cx)*scale + cx)),
                    int(round((y - cy)*scale + cy))))
    return out

# Sutherland–Hodgman clip against axis-aligned rectangle
def clip_polygon_with_rect(poly, x_min, y_min, x_max, y_max):
    def clip_edge(points, inside, intersect):
        if not points: return []
        out = []
        s = points[-1]
        for e in points:
            if inside(e):
                if inside(s): out.append(e)
                else: out.append(intersect(s, e)); out.append(e)
            else:
                if inside(s): out.append(intersect(s, e))
            s = e
        return out
    def inside_left(p):   return p[0] >= x_min
    def inside_right(p):  return p[0] <= x_max
    def inside_top(p):    return p[1] >= y_min
    def inside_bottom(p): return p[1] <= y_max
    def intersect_vert(p1, p2, xk):
        x1,y1=p1; x2,y2=p2
        if x2==x1: return (xk,y1)
        t=(xk-x1)/(x2-x1); y=y1+t*(y2-y1); return (xk,int(round(y)))
    def intersect_horiz(p1, p2, yk):
        x1,y1=p1; x2,y2=p2
        if y2==y1: return (x1,yk)
        t=(yk-y1)/(y2-y1); x=x1+t*(x2-x1); return (int(round(x)),yk)
    pts = [(float(x),float(y)) for x,y in poly]
    pts = clip_edge(pts, inside_left,  lambda a,b: intersect_vert(a,b,x_min))
    pts = clip_edge(pts, inside_right, lambda a,b: intersect_vert(a,b,x_max))
    pts = clip_edge(pts, inside_top,   lambda a,b: intersect_horiz(a,b,y_min))
    pts = clip_edge(pts, inside_bottom,lambda a,b: intersect_horiz(a,b,y_max))
    return [(int(round(x)), int(round(y))) for x,y in pts]

def load_pred_mat_for(img_path, pred_dir, suffix):
    stem = os.path.splitext(os.path.basename(img_path))[0]
    pred_path = os.path.join(pred_dir, stem + suffix)
    if not os.path.exists(pred_path):
        raise FileNotFoundError(f"Missing pred.mat for image: {pred_path}")
    d = loadmat(pred_path)
    arr = None
    for k in ("prediction","pred","label"):
        if k in d:
            arr = d[k]; break
    if arr is None:
        raise KeyError(f"No 'prediction' (or 'pred'/'label') key in {pred_path}")
    return np.array(arr).squeeze(), pred_path

# ---------- LOAD ----------
os.makedirs(OUTPUT_DIR, exist_ok=True)
image = Image.open(IMG_PATH).convert("RGB")
img_w, img_h = image.size
pred, pred_src = load_pred_mat_for(IMG_PATH, PRED_DIR, PRED_SUFFIX)

# handle potential transpose
if pred.shape != (img_h, img_w):
    if pred.T.shape == (img_h, img_w):
        pred = pred.T
    else:
        raise ValueError(f"pred shape {pred.shape} does not match image {(img_h,img_w)} for {pred_src}")
pred = pred.astype(np.int32)

np_img = np.array(image)
face_boxes = face_recognition.face_locations(np_img)
face_boxes = sorted(face_boxes, key=lambda b: b[3])  # left->right
if not face_boxes:
    raise RuntimeError("No faces detected in the image.")

draw = ImageDraw.Draw(image)

# Output file paths
stem = os.path.splitext(os.path.basename(IMG_PATH))[0]
out_png = os.path.join(OUTPUT_DIR, f"{stem}_aoi_preview.png")
out_csv = os.path.join(OUTPUT_DIR, f"{stem}_aoi_coordinates.csv")

rows = []  # CSV rows

# ---------- PER-FACE AOIs ----------
for i, box in enumerate(face_boxes, start=1):
    # expanded face rect to keep AOIs contained
    t, r, b, l = expand_box(box, img_w, img_h, ratio=EXPAND_FACE_BOX)
    face_mask = np.zeros((img_h, img_w), dtype=bool)
    face_mask[t:b, l:r] = True

    # segmentation masks, clipped to face rect
    eyes_mask = (pred == LBL_EYE)
    if INCLUDE_EYEBROWS_IN_EYES:
        eyes_mask |= (pred == LBL_BROW)
    mouth_mask = (pred == LBL_MOUTH)
    nose_mask  = (pred == LBL_NOSE)

    eyes_mask  &= face_mask
    mouth_mask &= face_mask
    nose_mask  &= face_mask

    # draw face rect + label (Face 1, Face 2, ...)
    draw.rectangle([l,t,r,b], outline="orange", width=3)
    draw.text((l+6, max(0, t-18)), f"Face {i}", fill=TEXT_COLOR)

    # ----- NOSE (scale AOI_SCALE_NOSE) -----
    nose_ccs = all_cc_masks(nose_mask, min_pixels=30)
    if nose_ccs:
        nose_ccs.sort(key=lambda m: m.sum(), reverse=True)
        nose_poly = polygon_from_mask(nose_ccs[0])
        if len(nose_poly) >= 3:
            nose_poly = enlarge_polygon(nose_poly, AOI_SCALE_NOSE)
            nose_poly = clip_polygon_with_rect(nose_poly, l, t, r, b)
            if len(nose_poly) >= 3:
                draw.line(nose_poly + [nose_poly[0]], fill=COLOR_NOSE, width=3)
                nx = min(x for x,_ in nose_poly); ny = min(y for _,y in nose_poly)
                draw.text((nx+5, ny+5), f"Nose_F{i}", fill=COLOR_NOSE)
                # save CSV rows
                for pi, (x,y) in enumerate(nose_poly, start=1):
                    rows.append({"AOI":"Nose","AOI_ID":f"Nose_F{i}","Face":i,
                                 "Component":1,"PointIndex":pi,"X":x,"Y":y})
    else:
        draw.text((l+8, t+8), "NOSE: none", fill=COLOR_NOSE)

    # ----- EYES (scale AOI_SCALE_EYES) -----
    eye_ccs = all_cc_masks(eyes_mask, min_pixels=30)
    if eye_ccs:
        # sort biggest first; enumerate components
        for k, m in enumerate(sorted(eye_ccs, key=lambda m: m.sum(), reverse=True), start=1):
            poly = polygon_from_mask(m)
            if len(poly) >= 3:
                poly = enlarge_polygon(poly, AOI_SCALE_EYES)
                poly = clip_polygon_with_rect(poly, l, t, r, b)
                if len(poly) >= 3:
                    draw.line(poly + [poly[0]], fill=COLOR_EYE, width=2)
                    ex = min(x for x,_ in poly); ey = min(y for _,y in poly)
                    draw.text((ex+4, ey+4), f"Eye_F{i}_{k}", fill=COLOR_EYE)
                    # save CSV rows
                    for pi, (x,y) in enumerate(poly, start=1):
                        rows.append({"AOI":"Eye","AOI_ID":f"Eye_F{i}_{k}","Face":i,
                                     "Component":k,"PointIndex":pi,"X":x,"Y":y})
    else:
        draw.text((l+8, t+28), "EYES: none", fill=COLOR_EYE)

    # ----- MOUTH (scale AOI_SCALE_MOUTH) -----
    mouth_ccs = all_cc_masks(mouth_mask, min_pixels=30)
    if mouth_ccs:
        mouth_ccs.sort(key=lambda m: m.sum(), reverse=True)
        poly = polygon_from_mask(mouth_ccs[0])
        if len(poly) >= 3:
            poly = enlarge_polygon(poly, AOI_SCALE_MOUTH)
            poly = clip_polygon_with_rect(poly, l, t, r, b)
            if len(poly) >= 3:
                draw.line(poly + [poly[0]], fill=COLOR_MOUTH, width=2)
                mx = min(x for x,_ in poly); my = min(y for _,y in poly)
                draw.text((mx+4, my+4), f"Mouth_F{i}", fill=COLOR_MOUTH)
                # save CSV rows
                for pi, (x,y) in enumerate(poly, start=1):
                    rows.append({"AOI":"Mouth","AOI_ID":f"Mouth_F{i}","Face":i,
                                 "Component":1,"PointIndex":pi,"X":x,"Y":y})
    else:
        draw.text((l+8, t+48), "MOUTH: none", fill=COLOR_MOUTH)

# ---------- SAVE PREVIEW & CSV ----------
image.save(out_png)

with open(out_csv, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["AOI","AOI_ID","Face","Component","PointIndex","X","Y"])
    writer.writeheader()
    writer.writerows(rows)

print(f"✅ Preview saved: {out_png}")
print(f"✅ AOI CSV saved: {out_csv}")

# ---------- Also show preview ----------
if SHOW_MODE.lower() == "pil":
    image.show()
else:
    try:
        import matplotlib.pyplot as plt
        plt.figure(); plt.imshow(image); plt.axis("off"); plt.show()
    except Exception as e:
        print("Matplotlib preview failed, falling back to PIL:", e); image.show()


✅ Preview saved: /Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ETbrave/aoi_outputs/BRAVE_adultF_adultF_1_aoi_preview.png
✅ AOI CSV saved: /Users/davrinarianda/Library/CloudStorage/Box-Box/PhD GGNB/Dissertation/Chapter 2/1. Eye-tracking assessment/ETbrave/aoi_outputs/BRAVE_adultF_adultF_1_aoi_coordinates.csv
